In [2]:
import re
import os

filePath = ".\\..\\design\\2layer\\rightside\\"
fileName = "rightside.kicad_pcb"
newFileName = "rightside_new.kicad_pcb"
if os.path.exists(filePath + fileName):
    print("File exists")

File exists


In [3]:
pattern = re.compile(r'uuid.+\n\s+\(at ([0-9\.]+) ([0-9\.]+)')
file = open(filePath + fileName, 'r')
fileText = file.read()
file.close()
matches = pattern.findall(fileText)
print(f"There are {len(matches)} matches")

There are 216 matches


In [4]:
# Mirror footprints
# Find largest x
maxX = 0
for match in matches:
    x = float(match[0])
    if x > maxX:
        maxX = x
print(f"Largest x is {maxX}")
# subtract largest x from all x and invert
newFileText = fileText
for match in matches:
    x = float(match[0])
    y = float(match[1])
    newX = maxX - x
    newFileText = newFileText.replace(f"(at {x} {y}", f"(at {newX} {y}")


Largest x is 159.692978


In [5]:
# Mirror edge cuts
edgecutLine = re.compile(r'gr_line\n\s+\(start ([0-9\.]+) ([0-9\.]+)\)\n\s+\(end ([0-9\.]+) ([0-9\.]+)')
matches = edgecutLine.findall(newFileText)
print(f"There are {len(matches)} edge cut lines")
for match in matches:
    x1 = float(match[0])
    y1 = float(match[1])
    x2 = float(match[2])
    y2 = float(match[3])
    newX1 = maxX - x1
    newX2 = maxX - x2
    newFileText = newFileText.replace(f"(start {x1} {y1}", f"(start {newX1} {y1}")
    newFileText = newFileText.replace(f"(end {x2} {y2}", f"(end {newX2} {y2}")

There are 8 edge cut lines


In [6]:
# Mirror circles
circlecutlines = re.compile(r'gr_circle\n\s+\(center ([0-9\.]+) ([0-9\.]+)\)\n\s+\(end ([0-9\.]+) ([0-9\.]+)')
matches = circlecutlines.findall(newFileText)
print(f"There are {len(matches)} circle cut lines")
for match in matches:
    x1 = float(match[0])
    y1 = float(match[1])
    x2 = float(match[2])
    y2 = float(match[3])
    newX1 = maxX - x1
    newX2 = maxX - x2
    newFileText = newFileText.replace(f"(center {x1} {y1}", f"(center {newX1} {y1}")
    newFileText = newFileText.replace(f"(end {x2} {y2}", f"(end {newX2} {y2}")


There are 1 circle cut lines


In [7]:
# Mirror vias
via = re.compile(r'via\n\s+\(at ([0-9\.]+) ([0-9\.]+)')
matches = via.findall(newFileText)
print(f"There are {len(matches)} vias")
for match in matches:
    x = float(match[0])
    y = float(match[1])
    newX = maxX - x
    newFileText = newFileText.replace(f"(at {x} {y}", f"(at {newX} {y}")

There are 0 vias


In [8]:
# mirror zones
zone = re.compile(r'\(polygon\n\s+\(pts\n\s+((?:\(xy [0-9\.]+ [0-9\.]+\)\s*)+)')
matches = zone.findall(newFileText)
print(f"There are {len(matches)} zones")
for match in matches:
    points = re.findall(r'\(xy ([0-9\.]+) ([0-9\.]+)\)', match)
    newPoints = []
    for point in points:
        x = float(point[0])
        y = float(point[1])
        newX = maxX - x
        newPoints.append(f"(xy {newX} {y})")
    newFileText = newFileText.replace(match, " ".join(newPoints))

There are 3 zones


In [9]:
file = open(filePath + newFileName, 'w')
file.write(newFileText)
file.close()